<a href="https://colab.research.google.com/github/barbaroja2000/rubric/blob/main/Rubric_Mistral_Large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluating Mistral Large with GPT-4

Uses:

*   Langsmith for custom rubrik datasets & Evaluation Framework
*   Mistal AI Platform for inference
*   GPT-4 To evaluate LLM output

---

Test:

*  provides a quality assesment 0-5 and a rationalle for the score
* The interactive table can be used to Check the model output against the GPT-4 assesment
* Possible improvment here is to be able to manually update the score returned from the model

---

Model Mistral-Large:

---

Refs:

https://mistral.ai/news/la-plateforme/
https://python.langchain.com/docs/langsmith/

---

Requires:
*   Test dataset in Langsmith: This notebook shows how to set this up  https://github.com/barbaroja2000/rubric/blob/main/Langsmith_Rubric_Dataset_Creator.ipynb
*   Mistral AI API Key
*   Langchain Key
*   OpenAI Key


In [1]:
!pip install -q langchain langsmith openai tiktoken mistralai

In [2]:
from google.colab import userdata
import os
import requests

os.environ['LANGCHAIN_ENDPOINT']= "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']= userdata.get('langchain_api_key')
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

In [3]:
mistralai_api_key = userdata.get('mistral_api_key')
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [4]:
mistral_client = MistralClient(api_key=mistralai_api_key)

In [5]:
project_name  = "Mistal AI - Large"
rubric_model = "mistral-large-latest"
rubric_temperature=0
evaluator_model="gpt-4-1106-preview"
evaluator_temperature=0
dataset_name = "General Rubric: Extended" #Use quick to test the runner

In [6]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

# create a string template for a System role
system_template = """
Please provide a detailed, accurate, and contextually relevant response to the following query.
Your answer should demonstrate a clear understanding of the subject matter, considering any logical,
ethical, historical, or cultural aspects involved. Ensure the response is tailored to the specific
requirements of the question, whether it involves analysis, explanation, creativity, or problem-solving.
"""

In [7]:
from langsmith import Client
client = Client()

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

def create_runnable():

  def get_response(input):
    # No Mistal LLM in Langchain yet
    chat_response = mistral_client.chat(
      model=rubric_model,
      messages=[ChatMessage(role="system", content=system_template), ChatMessage(role="user", content=input["input"]["question"])],
  )
    return chat_response.choices[0].message.content

  return RunnableMap({"input": RunnablePassthrough()}) | get_response

In [9]:
import re
from typing import Any, Optional

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import StringEvaluator

class QualityEvaluator(StringEvaluator):
    """An LLM-based quality  evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model=evaluator_model, temperature=evaluator_temperature)

        template = """ Evaluate the answer provided from 0 to 5, with 5 being the best quality answer, and 0 where the answer is false or not provided
        --------
        QUESTION: {input}
        --------
        ANSWER: {prediction}
        --------
        Consisely reason step by step about why the given score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = LLMChain.from_string(llm=llm, template=template)

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "quality"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain(
            dict(input=input, prediction=prediction), **kwargs
        )
        lines = evaluator_result["text"].strip().split("\n")
        reasoning = evaluator_result["text"].strip()
        score = lines[-1]
        if score is not None:
            score = float(score.strip())
        return {"score": score, "reasoning": reasoning}

In [10]:
import random
import string

def rando():
  # Generating a random 4-letter string
  random_string = ''.join(random.choices(string.ascii_letters, k=4))
  return random_string

In [11]:
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    input_key="question",
    custom_evaluators = [QualityEvaluator()]
)

project_metadata = {
  rubric_model : rubric_model,
  rubric_temperature : rubric_temperature,
  evaluator_model : evaluator_model,
  evaluator_temperature: evaluator_temperature
}

run = run_on_dataset(
    client=client,
    project_metadata = project_metadata,
    project_name = f"{project_name}-{rando()}",
    dataset_name=dataset_name,
    llm_or_chain_factory=create_runnable,
    evaluation=evaluation_config,
    concurrency_level=1, #This will bork if not provided or set to anything other than 1
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


View the evaluation results for project 'Mistal AI - Large-buTJ' at:
https://smith.langchain.com/o/8bebdbff-9433-40f6-b7e1-f013521a8100/datasets/dfedd53c-204a-4b69-9f64-51a61568a7fd/compare?selectedSessions=e2590de1-ee95-41fc-b1a3-58b829cfbe6d

View all tests for Dataset General Rubric: Extended at:
https://smith.langchain.com/o/8bebdbff-9433-40f6-b7e1-f013521a8100/datasets/dfedd53c-204a-4b69-9f64-51a61568a7fd
[>                                                 ] 0/123

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[------------------->                              ] 50/123

Error Type: MistralException, Message: Unexpected exception (ReadTimeout): The read operation timed out


...

[------------------------------------>             ] 91/123

Error Type: MistralException, Message: Unexpected exception (ReadTimeout): The read operation timed out


[---------------------------------------->         ] 100/123

Error Type: MistralException, Message: Unexpected exception (ReadTimeout): The read operation timed out


[------------------------------------------------->] 123/123

In [12]:
# Extracting the required information
extracted_data = []
for key, value in run['results'].items():
    output = value["output"] if "output" in value else None
    question = value['input']['question']
    for feedback_item in value['feedback']:
        if feedback_item.key == 'quality':
            quality = feedback_item.score
            feedback = feedback_item.comment
            extracted_data.append({
                'question': question,
                'output': output,
                'quality': quality,
                'feedback': feedback
            })

In [13]:
import pandas as pd
from google.colab import data_table
df = pd.DataFrame(extracted_data)

data_table.DataTable(df, include_index=False, num_rows_per_page=10)

,question,output,quality,feedback
0,"I WAS born in the year 1632, in the city of Yo...","Robinson Crusoe, born in 1632 in York, disrega...",4.0,The provided answer is a concise summary of th...
1,"I WAS born in the year 1632, in the city of Yo...","- The speaker, Robinson Kreutznaer, later know...",5.0,The provided answer offers a detailed and accu...
2,"Y curiosity, in a sense, was stronger than my ...","The main themes in this text are adventure, da...",3.0,The provided answer does a reasonable job of s...
3,Explain the role of a central bank in an econo...,A central bank plays a crucial role in managin...,5.0,The answer provided is comprehensive and uses ...
4,Describe the basics of how the stock market wo...,"Sure, I'd be happy to explain the basics of th...",5.0,"The answer provided is clear, concise, and acc..."
...,...,...,...,...
111,Analyze the emotional content of this diary en...,The emotional content of the diary entry appea...,5.0,The answer provided offers a thoughtful analys...
112,What is the emotional tone of this speech exce...,The emotional tone of this speech excerpt is o...,5.0,The answer provided is a comprehensive analysi...
113,Determine the emotional tone of this message: ...,"The emotional tone of the message is negative,...",5.0,The answer provided accurately identifies the ...
114,Analyze the emotion in this customer review: '...,The customer review you provided is overwhelmi...,5.0,The provided answer does an excellent job of a...


In [14]:
pd.to_numeric(df.quality).mean()

4.788793103448276

In [15]:
examples = list(client.list_examples(dataset_name=dataset_name))

In [16]:
# Creating a new DataFrame from the list
list_df = pd.DataFrame([{'question': item.inputs['question'], 'category': item.inputs['category']} for item in examples])

In [17]:
# Merging the DataFrames
merged_df = pd.merge(df, list_df, on='question')

In [18]:
# Group by 'category' and calculate the mean of 'quality' scores
avg_scores_per_category = merged_df.groupby('category')['quality'].mean()

print(avg_scores_per_category)

category
Categorization                           5.000000
Coding                                   4.880000
Creative Writing                         4.800000
Cultural and Contextual Understanding    4.850000
Emotion Analysis                         4.900000
Ethics Analysis                          4.857143
Fact Analysis                            5.000000
Logical Reasoning                        4.727273
Reading Comprehension                    4.444444
Reframing                                4.875000
Safety and Security                      4.500000
Summarization                            4.000000
Name: quality, dtype: float64
